<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW7/ResNet_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

import datetime

In [26]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 256
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers = 2)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False, num_workers = 2)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [27]:
class ResBlock (nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = self.batch_norm(out)
    return torch.relu(out + x)

In [28]:
class NetResDeep (nn.Module):
  def __init__(self, n_chans1=32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    #self.resblocks = nn.Sequential(*(n_blocks * [ResBlock(n_chans=n_chans1)]))
    self.resblocks = nn.Sequential(*[ResBlock(n_chans=n_chans1) for _ in range(n_blocks)])
    self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8*8*self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

In [29]:
# Switching to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [30]:
def train_and_validate(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
  device = next(model.parameters()).device

  for epoch in range(1, n_epochs+1):
    # Training Phase
    model.train()
    loss_train = 0.0
    correct_train = 0
    total_train = 0

    for imgs, labels in train_loader:
      imgs = imgs.to(device = device)
      labels = labels.to(device = device)

      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()

    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
      for imgs, labels in val_loader:
        imgs = imgs.to(device = device)
        labels = labels.to(device = device)

        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total_val += labels.shape[0]
        correct_val += int((predicted == labels).sum())

    avg_loss = loss_train / len(train_loader)
    acc_val = correct_val / total_val

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {:3d}, Training loss {:.4f}, Val Acc: {:.2f}'.format(
          datetime.datetime.now(), epoch, avg_loss, acc_val))

In [31]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 100

train_and_validate(n_epochs=n_epochs, optimizer=optimizer, model=model, loss_fn=loss_fn, train_loader=train_loader, val_loader=val_loader)

2025-11-26 00:07:33.687867 Epoch   1, Training loss 1.7073, Val Acc: 0.48
2025-11-26 00:08:39.265169 Epoch  10, Training loss 0.6359, Val Acc: 0.69
2025-11-26 00:09:53.919737 Epoch  20, Training loss 0.3864, Val Acc: 0.73
2025-11-26 00:11:06.434781 Epoch  30, Training loss 0.2152, Val Acc: 0.73
2025-11-26 00:12:18.483002 Epoch  40, Training loss 0.1406, Val Acc: 0.72
2025-11-26 00:13:30.603030 Epoch  50, Training loss 0.0836, Val Acc: 0.73
2025-11-26 00:14:42.093201 Epoch  60, Training loss 0.0485, Val Acc: 0.71
2025-11-26 00:15:54.955923 Epoch  70, Training loss 0.0589, Val Acc: 0.73
2025-11-26 00:17:06.718494 Epoch  80, Training loss 0.0208, Val Acc: 0.73
2025-11-26 00:18:19.884321 Epoch  90, Training loss 0.0012, Val Acc: 0.75
2025-11-26 00:19:31.109322 Epoch 100, Training loss 0.0267, Val Acc: 0.75


In [32]:
# Model Complexity
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(159594,
 [864,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  9216,
  32,
  32,
  65536,
  32,
  320,
  10])